## Benchmarking

In [1]:
import time
import autohf as hf

#### Atomic Norm

In [2]:
start = time.time()
hf.atomic_norm((1, 1, 1), [1.0, 2.0, 3.0], [3.0, 2.0, 1.0])
print("Time = {}s".format(time.time() - start))

Time = 0.00022602081298828125s


In [5]:
start = time.time()
hf.new_atomic_norm((1, 1, 1), [1.0, 2.0, 3.0], [3.0, 2.0, 1.0])
print("Time = {}s".format(time.time() - start))

[9. 6. 3. 6. 4. 2. 3. 2. 1.]
[2. 3. 4. 3. 4. 5. 4. 5. 6.]
Time = 0.0043680667877197266s
